Este es un modelo simple para analizar posiciones de ajedrez compatible con minimax y con poda alpha-beta

In [ ]:
import pandas as pd
import numpy as np
import chess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import pickle

df = pd.read_csv("evaluations.csv")

print("Proporción de mates:")
print(df["is_mate"].value_counts(normalize=True))

df.loc[df["is_mate"] == False, "cp"].dropna().hist(bins=100)
plt.title("Distribución de cp (sin mates)")
plt.show()

df.loc[df["is_mate"] == True, "mate_distance"].dropna().hist(bins=50)
plt.title("Distribución de mate_distance")
plt.show()


La mayoría de las posiciones no son mates y los mates son menos frecuentes por lo que los tratamos de forma diferente
preprocesado.


In [ ]:
def fen_to_features(fen):
    board = chess.Board(fen)
    feats = []

    for color in [chess.WHITE, chess.BLACK]:
        feats += [
            len(board.pieces(chess.PAWN, color)),
            len(board.pieces(chess.KNIGHT, color)),
            len(board.pieces(chess.BISHOP, color)),
            len(board.pieces(chess.ROOK, color)),
            len(board.pieces(chess.QUEEN, color)),
        ]

    feats.append(1 if board.turn == chess.WHITE else 0)
    return np.array(feats, dtype=np.float32)


def row_to_cp(row):
    if row["is_mate"]:
        return 20000.0 if row["mate_distance"] > 0 else -20000.0
    return row["cp"]


df = df.sample(n=200_000, random_state=0)

X = np.vstack(df["fen"].apply(fen_to_features))
y = np.clip(df.apply(row_to_cp, axis=1).to_numpy(), -2000, 2000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)


Las posiciones se representan mediante conteos de piezas por color y el turno de juego, voy a utilizar un modelo Ridge por su simplicidad y velocidad.

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

plt.hist(y_pred - y_test, bins=100)
plt.title("Distribución del error")
plt.show()

with open("ml_model.pkl", "wb") as f:
    pickle.dump(model, f)
